##Lesson 3: Working with Databases

#### if working with mysql or postgres we can use the sql extension to query directly from ipython notebook, and load into pandas dataframe

In [166]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [167]:
%sql mysql://root@localhost/bird

u'Connected: root@bird'

In [168]:
%config SqlMagic.autopandas=True

In [171]:
%sql SHOW TABLES;

5 rows affected.


,Tables_in_bird
0,checklists
1,locations
2,observers
3,sightings
4,species


In [182]:
%sql SELECT * FROM species LIMIT 10

10 rows affected.


,pk,taxonomic_order,category,common_name,scientific_name,subspecies_common_name,subspecies_scientific_name
0,1,1,species,Ostrich,Struthio camelus,,
1,2,3,issf,Ostrich,Struthio camelus,Ostrich (Common),Struthio camelus [camelus Group]
2,3,5,issf,Ostrich,Struthio camelus,Ostrich (Somali),Struthio camelus molybdophanes
3,4,7,species,Greater Rhea,Rhea americana,,
4,5,13,species,Lesser Rhea,Rhea pennata,,
5,6,17,issf,Lesser Rhea,Rhea pennata,Lesser Rhea (Darwin's),Rhea pennata pennata
6,7,18,species,Southern Cassowary,Casuarius casuarius,,
7,8,21,species,Emu,Dromaius novaehollandiae,,
8,9,29,species,North Island Brown Kiwi,Apteryx mantelli,,
9,10,30,species,Little Spotted Kiwi,Apteryx owenii,,


In [173]:
%sql SELECT COUNT(*) FROM SPECIES LIMIT 100

1 rows affected.


,COUNT(*)
0,7054


###3.1 SQLlite

In [183]:
# The sqlite3 module is used to work with the SQLite database.
import sqlite3 as lite

# Here you connect to the database. The `connect()` method returns a connection object.
con = lite.connect('getting_started.db')

In [187]:
print con.execute("SELECT name, state FROM cities;").fetchall()


[(u'New York City', u'NY'), (u'Boston', u'MA'), (u'Chicago', u'IL'), (u'Miami', u'FL'), (u'Dallas', u'TX'), (u'Seattle', u'WA'), (u'Portland', u'OR'), (u'San Francisco', u'CA'), (u'Los Angeles', u'CA'), (u'Washington', u'DC'), (u'Houston', u'TX'), (u'Washington', u'DC'), (u'Houston', u'TX'), (u'Washington', u'DC'), (u'Houston', u'TX'), (u'Washington', u'DC'), (u'Houston', u'TX'), (u'Washington', u'DC'), (u'Houston', u'TX')]


In [191]:
print con.execute("SELECT name, state FROM cities WHERE state='CA';").fetchall()

[(u'San Francisco', u'CA'), (u'Los Angeles', u'CA')]


In [190]:
print con.execute("SELECT name FROM cities LIMIT 2 OFFSET 3;").fetchall()

[(u'Miami',), (u'Dallas',)]


In [189]:
print con.execute("SELECT COUNT(*) FROM cities WHERE name LIKE 'San%' AND state='CA';").fetchall()

[(1,)]


In [188]:
con.execute("UPDATE cities SET state='Californ-I-A' WHERE state='CA';")
con.execute("UPDATE cities SET state='CA' WHERE state='Californ-I-A';")

####bonus exercise

rather than manually entering weather data from table on thinkful table, let's scrape that content and load it into a dataframe

In [21]:
import urllib2
url="https://courses.thinkful.com/data-001v2/assignment/1.3.1"
f=urllib2.urlopen(url)
html=[line.strip() for line in f]
print ' '.join(html)

<!DOCTYPE html> <html lang="en"> <head> <meta charset="UTF-8"> <meta name="viewport" content="width=device-width">  <title>Accounts | Thinkful</title> <meta name="description" content="Login to Thinkful and start learning today."> <link rel="stylesheet" href="/static/build/skeleton.css" />  <link rel="shortcut icon" href="//www.thinkful.com/favicon.ico" type="image/x-icon"> <link rel="icon" href="//www.thinkful.com/favicon.ico" type="image/x-icon">  <meta name="viewport" content="width=device-width, initial-scale=1">  <script type="text/javascript" src="//use.typekit.net/fxg2bgq.js"></script> <script type="text/javascript">try{Typekit.load();}catch(e){}</script>  <script type="text/javascript">  </script> </head> <body ng-app="hawk"> <div class="top-level-container"> <header class="login-header"> <img class="thinkful-logo" src="/static/tf-logo-web.svg"> </header>  <form class="login-form" method="POST" action="/login"> <label for="LoginInput"> Email Address </label> <input type="hidden

oops, this page has a login! I'll download the html directly from developer tools and save as sqlite_thinkful.html

In [34]:
with open('sqlite_thinkful.html', 'rU') as inputfile:
    html=[line.strip() for line in inputfile]
    html=' '.join(html)

In [138]:
from bs4 import BeautifulSoup
soup=BeautifulSoup(html)
for li in soup.find_all('li'):
    if "Add the following data to the table:" in li.get_text():
        data=li
data=data.find("pre").get_text()
header=data.split()[:8]
header.remove('Month')
header.remove('Month')
header.remove('High')
data=data.split()[8:]
data.insert(0,'0')

In [147]:
citydata,currcity=[],[]
sameline=True
for wordn,word in enumerate(data):
    if word=="2013":
        sameline=True
        city=[]
        i=1
        while not data[wordn-i].isdigit():
            city.append(data[wordn-i])
            i+=1
        city.reverse()
        city=' '.join(city)
        currcity=[city, word]
        citydata.append(currcity)
    elif len(currcity)!=0 and sameline:
        currcity.append(word)
        if word.isdigit():
            sameline=False
import pandas as pd
df=pd.DataFrame(columns=header, data=citydata)


In [202]:
df

,City,Year,Warm,Cold,Average
0,New York City,2013,July,January,62
1,Boston,2013,July,January,59
2,Chicago,2013,July,January,59
3,Miami,2013,August,January,84
4,Dallas,2013,July,January,77
5,Seattle,2013,July,January,61
6,Portland,2013,July,December,63
7,San Francisco,2013,September,December,64
8,Los Angeles,2013,September,December,75


In [157]:
querystr=[]
for index,row in df.iterrows():
    querystr.append(str(tuple([str(val) for val in row.values])))
querystr="INSERT INTO weather (city, year, warm_month, cold_month, average_high) VALUES"+', '.join(querystr)

In [192]:
con.execute(querystr)

###3.2 SQL-python

####connecting to db

In [1]:
import sqlite3 as lite
con = lite.connect('getting_started.db')

with con:
  # From the connection, you get a cursor object. The cursor is what goes over the records that result from a query.
  cur = con.cursor()    
  cur.execute('SELECT SQLITE_VERSION()')
  # You're fetching the data from the cursor object. Because you're only fetching one record, you'll use the `fetchone()` method. If fetching more than one record, use the `fetchall()` method.
  data = cur.fetchone()
  # Finally, print the result.
  print "SQLite version: %s" % data

SQLite version: 3.8.3


####writing to db

In [163]:
import sqlite3 as lite

con = lite.connect('getting_started.db')

# Inserting rows by passing values directly to `execute()`
with con:

    cur = con.cursor()
    cur.execute("INSERT INTO cities VALUES('Washington', 'DC')")
    cur.execute("INSERT INTO cities VALUES('Houston', 'TX')")
    cur.execute("INSERT INTO weather VALUES('Washington', 2013, 'July', 'January', 65)")
    cur.execute("INSERT INTO weather VALUES('Houston', 2013, 'July', 'January', 70)")

###3.3 Joining and Filtering Data

In [206]:
query='''SELECT DISTINCT name, state, year, warm_month, cold_month 
        FROM cities 
        INNER JOIN weather 
            ON name = city;
'''
cur.execute(query).fetchall()

[(u'New York City', u'NY', 2013, u'July', u'January'),
 (u'Boston', u'MA', 2013, u'July', u'January'),
 (u'Chicago', u'IL', 2013, u'July', u'January'),
 (u'Miami', u'FL', 2013, u'August', u'January'),
 (u'Dallas', u'TX', 2013, u'July', u'January'),
 (u'Seattle', u'WA', 2013, u'July', u'January'),
 (u'Portland', u'OR', 2013, u'July', u'December'),
 (u'San Francisco', u'CA', 2013, u'September', u'December'),
 (u'Los Angeles', u'CA', 2013, u'September', u'December'),
 (u'Washington', u'DC', 2013, u'July', u'January'),
 (u'Houston', u'TX', 2013, u'July', u'January')]

In [226]:
cur.execute("SELECT state, ROUND(AVG(average_high),2) FROM weather INNER JOIN cities on city=name GROUP BY state;").fetchall()

[(u'CA', 69.5),
 (u'DC', 65.0),
 (u'FL', 84.0),
 (u'IL', 59.0),
 (u'MA', 59.0),
 (u'NY', 62.0),
 (u'OR', 63.0),
 (u'TX', 70.33),
 (u'WA', 61.0)]

In [230]:
print cur.execute("SELECT warm_month, AVG(average_high) FROM weather GROUP BY warm_month;").fetchall()
print
print cur.execute("SELECT city, average_high FROM weather ORDER BY average_high;").fetchall()
print
print cur.execute("SELECT city, average_high FROM weather ORDER BY average_high DESC;").fetchall()
print
print cur.execute("SELECT state, ROUND(AVG(average_high),2) FROM weather INNER JOIN cities on city=name GROUP BY state ORDER BY AVG(average_high) DESC;").fetchall()


[(u'August', 84.0), (u'July', 65.78571428571429), (u'September', 69.5)]

[(u'Boston', 59), (u'Chicago', 59), (u'Seattle', 61), (u'New York City', 62), (u'Portland', 63), (u'San Francisco', 64), (u'Washington', 65), (u'Washington', 65), (u'Washington', 65), (u'Washington', 65), (u'Houston', 70), (u'Houston', 70), (u'Houston', 70), (u'Houston', 70), (u'Los Angeles', 75), (u'Dallas', 77), (u'Miami', 84)]

[(u'Miami', 84), (u'Dallas', 77), (u'Los Angeles', 75), (u'Houston', 70), (u'Houston', 70), (u'Houston', 70), (u'Houston', 70), (u'Washington', 65), (u'Washington', 65), (u'Washington', 65), (u'Washington', 65), (u'San Francisco', 64), (u'Portland', 63), (u'New York City', 62), (u'Seattle', 61), (u'Boston', 59), (u'Chicago', 59)]

[(u'FL', 84.0), (u'TX', 70.33), (u'CA', 69.5), (u'DC', 65.0), (u'OR', 63.0), (u'NY', 62.0), (u'WA', 61.0), (u'IL', 59.0), (u'MA', 59.0)]


In [231]:

print cur.execute("SELECT warm_month, AVG(average_high) FROM weather GROUP BY warm_month HAVING AVG(average_high) > 65;").fetchall()
print
print cur.execute("SELECT state, ROUND(AVG(average_high),2) FROM weather INNER JOIN cities on city=name GROUP BY state HAVING AVG(average_high)>65 ORDER BY AVG(average_high) DESC;").fetchall()


[(u'August', 84.0), (u'July', 65.78571428571429), (u'September', 69.5)]

[(u'FL', 84.0), (u'TX', 70.33), (u'CA', 69.5)]


###EXTRA: SQL to Pandas Comparison

http://pandas.pydata.org/pandas-docs/stable/comparison_with_sql.html